In [9]:
import pandas as pd 
import numpy as np 
import implicit
from scipy.sparse import csr_matrix


In [13]:
hot_encoded = pd.read_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/balanced_ground_truth_hot_vector.csv")
sparse_matrix = csr_matrix(hot_encoded.iloc[:,1:].values)

In [59]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=100, calculate_training_loss = True)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sparse_matrix)

In [92]:
np.sum(np.abs(np.matmul(model.item_factors,model.user_factors.T) - hot_encoded.iloc[:,1:].values))

15685.0

In [90]:
track_embeddings = pd.DataFrame(model.item_factors)
track_embeddings["song_id"] = hot_encoded.song_id
cols = track_embeddings.columns.tolist()
cols = cols[-1:] + cols[:-1]
track_embeddings = track_embeddings[cols]
track_embeddings.to_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/tracks_embeddings_groundtruth.csv",index = False)

In [97]:
# Getting training validation and test splits
trainings = pd.read_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/train_ground_truth.csv")
validations = pd.read_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/validation_ground_truth.csv")
test = pd.read_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/test_ground_truth.csv")
train_embeddings = track_embeddings[track_embeddings.song_id.isin(trainings.song_id)]
valid_embeddings = track_embeddings[track_embeddings.song_id.isin(validations.song_id)]
test_embeddings = track_embeddings[track_embeddings.song_id.isin(test.song_id)]

    train_embeddings.to_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/train_embeddings_groundtruth.csv",index = False)
valid_embeddings.to_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/validation_embeddings_groundtruth.csv",index = False)
test_embeddings.to_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/test_embeddings_groundtruth.csv",index = False)

In [91]:
context_embeddings = pd.DataFrame(model.user_factors)
context_embeddings["context"] = hot_encoded.columns[1:]
cols = context_embeddings.columns.tolist()
cols = cols[-1:] + cols[:-1]
context_embeddings = context_embeddings[cols]
context_embeddings.to_csv("/home/karim/Documents/research/sourceCode/context_classification_cnn/GroundTruth/context_embeddings.csv",index = False)

In [100]:
predicts = np.loadtxt("/home/karim/Documents/research/exeriment_results_server_backup/experiments_results/embeddings_C4/2019-09-11_15-18-58/predicted_embedings.out",delimiter=',')

In [102]:
np.matmul(predicts,model.user_factors.T)

array([[0.37872151, 0.57213471, 0.28169027, ..., 0.38025338, 0.41720732,
        0.23606426],
       [0.37872151, 0.57213471, 0.28169027, ..., 0.38025338, 0.41720732,
        0.23606426],
       [0.37872151, 0.57213471, 0.28169027, ..., 0.38025338, 0.41720732,
        0.23606426],
       ...,
       [0.37872151, 0.57213471, 0.28169027, ..., 0.38025338, 0.41720732,
        0.23606426],
       [0.37872151, 0.57213471, 0.28169027, ..., 0.38025338, 0.41720732,
        0.23606426],
       [0.37872999, 0.57214741, 0.28169725, ..., 0.38026331, 0.41721609,
        0.2360718 ]])

In [103]:
context_embeddings = pd.read_csv(
            os.path.join(SOURCE_PATH, "GroundTruth/context_embeddings.csv"))


NameError: name 'os' is not defined